# OpenCV_Image_Area_Processing

In [2]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

## 블러링 - 이미지를 점진적으로 변하게 해서 부드러운 느낌이 나도록 하는 것

### 직접 구현

In [9]:
#이미지 가져오기 - 흑백으로 변환해서 가져오기
image = cv2.imread("./data/filter_blur.jpg", cv2.IMREAD_GRAYSCALE)
#이미지 출력
cv2.imshow("image", image)

#블러링을 적용할 마스크 행렬 - 홀수의 정방행렬로 생성
# 3 X 3 행렬로 생성
data = [1/9, 1/9, 1/9,
       1/9, 1/9, 1/9,
       1/9, 1/9, 1/9]
mask = np.array(data, np.float32).reshape(3, 3)
#print(mask)

##회선 수행 - 화소에 직접 접근

#행과 열의 수 구하기
rows, cols = image.shape[:2]

#원본의 행 과 열의 크기로 회선 결과를 저장할 행렬을 생성
dst = np.zeros((rows, cols), np.float32)

#마스크 중심 좌표 찾아오기
xcenter, ycenter = mask.shape[1]//2, mask.shape[0]//2

#회선 수행 - 맨 왼쪽 과 오른쪽은 회선을 돌릴 수가 없으므로 0부터 시작하면 안됨
for i in range(ycenter, rows - ycenter):
    for j in range(xcenter, cols - xcenter):
        sum = 0.0
        for u in range(mask.shape[0]):
            for v in range(mask.shape[1]):
                y = i + u - ycenter
                x = j + v - xcenter
                sum += image[y, x] * mask[u, v]
        dst[i, j] = sum
        
cv2.imshow("pixel", dst.astype("uint8"))       

#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()

## Shapening: 이미지를 날카롭게 만드는 작업

In [16]:
#이미지 가져오기 - 흑백으로 변환해서 가져오기
image = cv2.imread("./data/filter_sharpen.jpg", cv2.IMREAD_GRAYSCALE)
#이미지 출력
cv2.imshow("image", image)

#2개의 마스크 생성
data1 = [0, -1, 0, 
      -1, 5, -1,
      0, -1, 0]

data2 = [-1, -1, -1, 
      -1, 9, -1,
      -1, -1, -1]
mask1 = np.array(data1, np.float32).reshape(3, 3)
mask2 = np.array(data2, np.float32).reshape(3, 3)

#shapening을 위한 회선 수행 함수
def filter(image, mask):
    #행과 열의 수 구하기
    rows, cols = image.shape[:2]

    #원본의 행 과 열의 크기로 회선 결과를 저장할 행렬을 생성
    dst = np.zeros((rows, cols), np.float32)

    #마스크 중심 좌표 찾아오기
    xcenter, ycenter = mask.shape[1]//2, mask.shape[0]//2
    #회선 수행 - 맨 왼쪽 과 오른쪽은 회선을 돌릴 수가 없으므로 0부터 시작하면 안됨
    for i in range(ycenter, rows - ycenter):
        for j in range(xcenter, cols - xcenter):
            sum = 0.0
            for u in range(mask.shape[0]):
                for v in range(mask.shape[1]):
                    y = i + u - ycenter
                    x = j + v - xcenter
                    sum += image[y, x] * mask[u, v]
            dst[i, j] = sum
    return dst

#샤프닝을 수행 - 가중치의 합이 1이 넘기 때문에 255가 넘는 수가 나올 수 있음
sharpen1 = filter(image, mask1)
sharpen2 = filter(image, mask2)
#정규화 수행
sharpen1 = cv2.convertScaleAbs(sharpen1)
sharpen2 = cv2.convertScaleAbs(sharpen2)

cv2.imshow("sharpen1", sharpen1)
cv2.imshow("sharpen2", sharpen2)

#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()

## 필터 함수 적용

### 이미지 흐리게 하기

In [19]:
#이미지 가져오기 - 흑백으로 변환해서 가져오기
image = cv2.imread("./data/plane.jpg", cv2.IMREAD_GRAYSCALE)

#커널 생성
kernel = np.ones((5,5)) / 25.0
print(kernel)

image_kernel = cv2.filter2D(image, -1, kernel)


#이미지 출력
cv2.imshow("image", image)
cv2.imshow("image_kernel", image_kernel)

#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]]


### 이미지 선명하게 하기

In [20]:
#이미지 가져오기 - 흑백으로 변환해서 가져오기
image = cv2.imread("./data/plane.jpg", cv2.IMREAD_GRAYSCALE)

#커널 생성
kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
print(kernel)

image_kernel = cv2.filter2D(image, -1, kernel)


#이미지 출력
cv2.imshow("image", image)
cv2.imshow("image_kernel", image_kernel)

#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 0 -1  0]
 [-1  5 -1]
 [ 0 -1  0]]


### 로버츠 마스크 - 대각 방향으로 1 과 -1을 배치

In [26]:
def filter(image, mask):
    #행과 열의 수 구하기
    rows, cols = image.shape[:2]

    #원본의 행 과 열의 크기로 회선 결과를 저장할 행렬을 생성
    dst = np.zeros((rows, cols), np.float32)

    #마스크 중심 좌표 찾아오기
    xcenter, ycenter = mask.shape[1]//2, mask.shape[0]//2
    #회선 수행 - 맨 왼쪽 과 오른쪽은 회선을 돌릴 수가 없으므로 0부터 시작하면 안됨
    for i in range(ycenter, rows - ycenter):
        for j in range(xcenter, cols - xcenter):
            sum = 0.0
            for u in range(mask.shape[0]):
                for v in range(mask.shape[1]):
                    y = i + u - ycenter
                    x = j + v - xcenter
                    sum += image[y, x] * mask[u, v]
            dst[i, j] = sum
    return dst

#Robers Mask를 위한 함수
def differential(image, data1, data2):
    #일차원 리스트를 행렬로 변환
    mask1 = np.array(data1, np.float32).reshape(3, 3)
    mask2 = np.array(data2, np.float32).reshape(3, 3)
    
    #회선 수행
    dst1 = filter(image, mask1)
    dst2 = filter(image, mask2)
    
    #회선을 수행한 행렬의 크기 계산
    dst = cv2.magnitude(dst1, dst2)
    
    #음수가 있으면 양수로 변환
    dst1, dst2 = np.abs(dst1), np.abs(dst2)
    
    #255가 넘거나 0보다 작은 수가 있으면 잘라내기
    dst = np.clip(dst, 0, 255).astype("uint8")
    dst1 = np.clip(dst1, 0, 255).astype("uint8")
    dst2 = np.clip(dst2, 0, 255).astype("uint8")
    
    return dst, dst1, dst2
    

#이미지 가져오기 - 흑백으로 변환해서 가져오기
image = cv2.imread("./data/edge.jpg", cv2.IMREAD_GRAYSCALE)


#로버츠 마스크를 생성
data1 = [-1, 0, 0, 
        0, 1, 0, 
        0, 0, 0]

data2 = [0, 0, -1, 
        0, 1, 0, 
        0, 0, 0]

#로버츠 마스크 적용
dst, dst1, dst2 = differential(image, data1, data2)

#이미지 출력
cv2.imshow("image", image)
cv2.imshow("dst", dst)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)


#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()

### Prewitt Mask - 수직이나 수평 방향으로 -1 과 1을 배치

In [27]:
def filter(image, mask):
    #행과 열의 수 구하기
    rows, cols = image.shape[:2]

    #원본의 행 과 열의 크기로 회선 결과를 저장할 행렬을 생성
    dst = np.zeros((rows, cols), np.float32)

    #마스크 중심 좌표 찾아오기
    xcenter, ycenter = mask.shape[1]//2, mask.shape[0]//2
    #회선 수행 - 맨 왼쪽 과 오른쪽은 회선을 돌릴 수가 없으므로 0부터 시작하면 안됨
    for i in range(ycenter, rows - ycenter):
        for j in range(xcenter, cols - xcenter):
            sum = 0.0
            for u in range(mask.shape[0]):
                for v in range(mask.shape[1]):
                    y = i + u - ycenter
                    x = j + v - xcenter
                    sum += image[y, x] * mask[u, v]
            dst[i, j] = sum
    return dst

#Prewitt Mask를 위한 함수
def differential(image, data1, data2):
    #일차원 리스트를 행렬로 변환
    mask1 = np.array(data1, np.float32).reshape(3, 3)
    mask2 = np.array(data2, np.float32).reshape(3, 3)
    
    #회선 수행
    dst1 = filter(image, mask1)
    dst2 = filter(image, mask2)
    
    #회선을 수행한 행렬의 크기 계산
    dst = cv2.magnitude(dst1, dst2)
    
    #음수가 있으면 양수로 변환
    dst1, dst2 = np.abs(dst1), np.abs(dst2)
    
    #255가 넘거나 0보다 작은 수가 있으면 잘라내기
    dst = np.clip(dst, 0, 255).astype("uint8")
    dst1 = np.clip(dst1, 0, 255).astype("uint8")
    dst2 = np.clip(dst2, 0, 255).astype("uint8")
    
    return dst, dst1, dst2
    

#이미지 가져오기 - 흑백으로 변환해서 가져오기
image = cv2.imread("./data/edge.jpg", cv2.IMREAD_GRAYSCALE)


#로버츠 마스크를 생성
data1 = [-1, 0, 1, 
        -1, 0, 1, 
        -1, 0, 1]

data2 = [-1, -1, -1, 
        0, 0, 0, 
        1, 1, 1]

#프리윗 마스크 적용
dst, dst1, dst2 = differential(image, data1, data2)

#이미지 출력
cv2.imshow("image", image)
cv2.imshow("dst", dst)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)


#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()

### 소벨 마스크

In [31]:
image = cv2.imread('./data/rect.jpg', cv2.IMREAD_GRAYSCALE)

#소벨 마스크 적용
gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)

#극좌표로 변환
mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees = True)

#이미지 이진화
ret, edge = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY)

#이미지 출력
cv2.imshow("image", image)

cv2.imshow("gx", gx)
cv2.imshow("gy", gy)
cv2.imshow("edge", edge)

#키보드를 누르면 종료하는 코드
cv2.waitKey(0)
cv2.destroyAllWindows()